In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import shutil
import os


S = np.load('data/S20_30.npy')
SS = np.array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
               [-1, +1, -1, -1, -1, -1, -1, +1, -1, -1, -1, -1, -1, +1, -1, -1, -1, -1],
               [-1, -1, -1, -1, -1, -1, -1, -1, -1, +1, +1, +1, -1, -1, -1, -1, -1, -1],
               [-1, +1, -1, -1, -1, -1, -1, +1, -1, +1, +1, +1, -1, +1, -1, -1, -1, -1]])

Variables = np.load('data/Variables.npy')


# pr and prr are the shortcut for cosh functions.
def pr(b, w, s1, s2, s3, s4):
    result = torch.cosh(b[0] + 1j * (w[0] * s1 + w[1] * s2 + w[2] * s3 + w[3] * s4))
    return result
def prr(b, b1, b2, w, s1, s2, s3):
    result = torch.cosh(1j * b + (b1 + b2 * 1j) + 1j * w * (s1 + s2 + s3))
    return result
def prrr(b, b1, b2, w, s1, s2, s3, s4, s5, s6):
    result = torch.cosh(1j * b + (b1 + b2 * 1j) + 1j * w * (s1 + s2 + s3 + s4 + s5 + s6))
    return result


# ph is the phase function.
def ph(bl, phase, wl, al, bv, wv, bf, wf, s):
    result = (np.exp(1j * GL[0]) * np.exp(1j * GL[1]) * np.exp(1j * GL[2])
              * torch.exp(1j * al[0] * (s[0]+s[1]+s[2])) * torch.exp(1j * al[1] * (s[3]+s[9]+s[15]))
              * torch.exp(1j * al[2] * (s[0]+s[1]+s[2]+s[3]+s[9]+s[15]))
              * prr(bl[0], phase[0], phase[3], wl[0], s[0], s[1], s[2])
              * prr(bl[1], phase[1], phase[4], wl[1], s[3], s[9], s[15])
              * prrr(bl[2], phase[2], phase[5], wl[2], s[0], s[1], s[2], s[3], s[9], s[15])
              * pr(bv, wv, s[0], s[3], s[2], s[15]) * pr(bv, wv, s[1], s[4], s[0], s[16])
              * pr(bv, wv, s[2], s[5], s[1], s[17]) * pr(bv, wv, s[6], s[9], s[8], s[3])
              * pr(bv, wv, s[7], s[10], s[6], s[4]) * pr(bv, wv, s[8], s[11], s[7], s[5])
              * pr(bv, wv, s[12], s[15], s[14], s[9]) * pr(bv, wv, s[13], s[16], s[12], s[10])
              * pr(bv, wv, s[14], s[17], s[13], s[11]) * pr(bf, wf, s[4], s[6], s[3], s[0])
              * pr(bf, wf, s[5], s[7], s[4], s[1]) * pr(bf, wf, s[3], s[8], s[5], s[2])
              * pr(bf, wf, s[10], s[12], s[9], s[6]) * pr(bf, wf, s[11], s[13], s[10], s[7])
              * pr(bf, wf, s[9], s[14], s[11], s[8]) * pr(bf, wf, s[16], s[0], s[15], s[12])
              * pr(bf, wf, s[17], s[1], s[16], s[13]) * pr(bf, wf, s[15], s[2], s[17], s[14]))
    return result


# nor is the normalize function.
def nor(bl, phase, wl, al, bv, wv, bf, wf):
    result = 0
    for i in range(len(S)):
      result += abs(ph(bl, phase, wl, al, bv, wv, bf, wf, S[i]))
    return result


# The 9 vertex operators flip qubits.
A1 = np.diag([-1, +1, -1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, +1])
A2 = np.diag([-1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1])
A3 = np.diag([+1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1])
A4 = np.diag([+1, +1, +1, -1, +1, +1, -1, +1, -1, -1, +1, +1, +1, +1, +1, +1, +1, +1])
A5 = np.diag([+1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1])
A6 = np.diag([+1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1])
A7 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, +1, -1, +1, -1, -1, +1, +1])
A8 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1])
A9 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1])
S1 = np.dot(S, A1)
S2 = np.dot(S, A2)
S3 = np.dot(S, A3)
S4 = np.dot(S, A4)
S5 = np.dot(S, A5)
S6 = np.dot(S, A6)
S7 = np.dot(S, A7)
S8 = np.dot(S, A8)
S9 = np.dot(S, A9)

# lam is the multiplier to amplify the choice of ground state
def criterion(bl, phase, wl, al, bv, wv, bf, wf):
    V = torch.empty((18*N+3,), dtype=torch.complex64)
    for k in range(N):
        sub = ph(bl, phase, wl, al, bv, wv, bf, wf, S[k])
        one = ph(bl, phase, wl, al, bv, wv, bf, wf, SS[0])
        V[k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S1[k]) - sub
        V[1*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S2[k]) - sub
        V[2*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S3[k]) - sub
        V[3*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S4[k]) - sub
        V[4*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S5[k]) - sub
        V[5*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S6[k]) - sub
        V[6*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S7[k]) - sub
        V[7*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S8[k]) - sub
        V[8*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S9[k]) - sub
        V[9*N+k] = (S[k][4]*S[k][6]*S[k][3]*S[k][0]-1) * sub
        V[10*N+k] = (S[k][5]*S[k][7]*S[k][4]*S[k][1]-1) * sub
        V[11*N+k] = (S[k][3]*S[k][8]*S[k][5]*S[k][2]-1) * sub
        V[12*N+k] = (S[k][10]*S[k][12]*S[k][9]*S[k][6]-1) * sub
        V[13*N+k] = (S[k][11]*S[k][13]*S[k][10]*S[k][7]-1) * sub
        V[14*N+k] = (S[k][9]*S[k][14]*S[k][11]*S[k][8]-1) * sub
        V[15*N+k] = (S[k][16]*S[k][0]*S[k][15]*S[k][12]-1) * sub
        V[16*N+k] = (S[k][17]*S[k][1]*S[k][16]*S[k][13]-1) * sub
        V[17*N+k] = (S[k][15]*S[k][2]*S[k][17]*S[k][14]-1) * sub
        V[18*N+0] = lam * (ph(bl, phase, wl, al, bv, wv, bf, wf, SS[1])-2*one)
        V[18*N+1] = lam * (ph(bl, phase, wl, al, bv, wv, bf, wf, SS[2])-3*one)
        V[18*N+2] = lam * (ph(bl, phase, wl, al, bv, wv, bf, wf, SS[3])-4*one)
    v = torch.norm(V)/nor(bl, phase, wl, al, bv, wv, bf, wf)
    return v

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start_time = time.time()
(ss, tt, lam, N) = (0.03, 50, 1.0, len(S))
BF = torch.tensor([0.0])
WF = torch.tensor([np.pi/4, np.pi/4, np.pi/4, np.pi/4])
GL = torch.tensor([np.pi/4, np.pi/4, np.pi/2])
BL = torch.tensor([np.pi/4, np.pi/4, 0.0])
WL = torch.tensor([np.pi/4, np.pi/4, np.pi/4])
AL = torch.tensor([np.pi/4, np.pi/4, np.pi/4])
All = []
Select = []
for k in range(0, 99):
    IN = Variables[k]
    BV = torch.tensor([IN[0]], requires_grad=True)
    WV = torch.tensor([IN[1], IN[2], IN[3], IN[4]], requires_grad=True)
    Phase = torch.tensor([IN[5], IN[6], IN[7], IN[8], IN[9], IN[10]], requires_grad=True)
    optimizer = torch.optim.Adam([BV, WV, Phase], lr=ss)
    for h in range(tt):
        cost = criterion(BL, Phase, WL, AL, BV, WV, BF, WF)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
        All.append(IN.tolist())
    print("%d: (%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f)->(%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f), cost=%.2e" % (k, IN[0].tolist(), IN[1].tolist(), IN[2].tolist(), IN[3].tolist(),IN[4].tolist(), IN[5].tolist(), IN[6].tolist(), IN[7].tolist(),IN[8].tolist(), IN[9].tolist(), IN[10].tolist(),BV[0].tolist(), WV[0].tolist(), WV[1].tolist(), WV[2].tolist(), WV[3].tolist(),Phase[0].tolist(),Phase[1].tolist(),Phase[2].tolist(),Phase[3].tolist(),Phase[4].tolist(),Phase[5].tolist(),cost))
    if cost < 0.01:
        Select.append(IN.tolist())
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")
print(f"Select:{Select}")


0: (-2.95,-3.03,-0.86,-1.38,-0.06,-0.25,-0.02,0.94,-0.51,-0.98,1.55)->(-3.33,-3.18,-1.19,-1.53,-0.39,-0.24,-0.23,1.43,0.18,-0.28,0.92), cost=1.17e-01
1: (-2.41,-2.12,1.90,2.97,3.04,-1.08,-1.44,-0.70,0.35,1.09,0.48)->(-2.16,-1.75,1.72,3.32,2.99,-0.34,-1.00,-1.24,-0.01,-0.07,0.07), cost=5.10e-02
2: (-2.44,-1.59,3.05,2.59,1.99,-1.48,-1.57,-0.72,0.29,1.29,0.35)->(-2.79,-1.32,2.91,2.87,1.81,-0.39,-1.87,-0.74,-0.00,0.11,-0.01), cost=6.30e-02
3: (2.27,2.14,2.51,-2.64,2.97,-0.86,1.10,-1.39,1.37,1.27,-1.03)->(1.54,2.38,2.05,-2.40,2.67,-1.36,1.47,-0.83,0.28,0.26,-0.09), cost=2.07e-01
4: (0.63,-1.38,1.29,-2.16,3.08,0.03,-1.03,1.57,-0.88,1.10,0.79)->(1.30,-1.08,1.56,-2.07,3.14,0.32,-1.39,1.89,-0.28,0.28,0.33), cost=1.65e-01
5: (-1.70,2.70,2.79,1.35,1.33,0.98,-0.69,-0.42,0.33,-0.03,-1.16)->(-2.06,2.24,3.09,0.88,1.62,0.49,-0.81,-1.02,-0.06,-0.04,-0.55), cost=5.27e-02
6: (1.24,2.21,0.52,0.52,-3.00,-0.09,-1.54,-0.47,-0.77,1.40,-1.11)->(2.12,2.61,0.16,0.54,-3.31,-0.10,-2.04,-0.76,-0.03,0.42,-0.61), cos

In [ ]:
26: (-0.50,3.14,0.19,-1.36,2.27,-1.29,-0.95,-1.09,-0.44,0.87,0.26)->(-0.01,3.12,0.00,-1.58,1.51,-0.66,-0.45,-1.17,0.00,0.00,0.01), cost=7.20e-03